<a href="https://colab.research.google.com/github/hollipista/MetaCortex-AAEE/blob/main/02_MetaCortex_AAEE_anal_linked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.api as sm
import gc
import requests

# Database handler
#from sqlalchemy import create_engine, text, MetaData, Table, Column, Integer, Date, String, FLOAT

# Necessary for compatibility
#from sqlalchemy.dialects.postgresql.base import PGDialect
#PGDialect._get_server_version_info = lambda *args: (9, 2)

# Necessary for standard t-stat transformation
from statistics import NormalDist

In [ ]:
# előkészített adatok letöltése és kicsomagolása
!wget -O CRSP_data_ready.csv.zip https://www.dropbox.com/scl/fi/nut6nhrndvy5k2cnjt1pb/CRSP_data_ready.csv.zip?rlkey=1zvidp8l8s60pyqjlskh0o8pk&dl=0
!wget -O sp500_data_ready.csv.zip https://www.dropbox.com/scl/fi/di2e7ug7c96wekqg450vw/sp500_data_ready.csv.zip?rlkey=42byzuo3p1m4rhqsxrzsx5kwf&dl=0
!wget -O ff3_data_ready.csv.zip https://www.dropbox.com/scl/fi/nc9weel4i2hhfpa2tf1qp/ff3_data_ready.csv.zip?rlkey=rmz7g2522c5o0tx0aegdfrewf&dl=0

!unzip -n /content/CRSP_data_ready.csv.zip
!unzip -n /content/sp500_data_ready.csv.zip
!unzip -n /content/ff3_data_ready.csv.zip

--2023-11-30 19:31:29--  https://www.dropbox.com/scl/fi/nut6nhrndvy5k2cnjt1pb/CRSP_data_ready.csv.zip?rlkey=1zvidp8l8s60pyqjlskh0o8pk
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3db145b7c034fad482258d867c.dl.dropboxusercontent.com/cd/0/inline/CIgchmEmi1OeeOhQsWFJxbvCKL3R9HUGOuu-B1KGIy-gc4ySw7cL4OFqTbuiV1jBJWjgQnoKDU5cOZsStYwDdhVk4juwE2Lig05VrbkM4SGv7NHeR0eke0nOT3ltl6VVEyjIC46gawbndxBLtshPB8QH/file# [following]
--2023-11-30 19:31:30--  https://uc3db145b7c034fad482258d867c.dl.dropboxusercontent.com/cd/0/inline/CIgchmEmi1OeeOhQsWFJxbvCKL3R9HUGOuu-B1KGIy-gc4ySw7cL4OFqTbuiV1jBJWjgQnoKDU5cOZsStYwDdhVk4juwE2Lig05VrbkM4SGv7NHeR0eke0nOT3ltl6VVEyjIC46gawbndxBLtshPB8QH/file
Resolving uc3db145b7c034fad482258d867c.dl.dropboxusercontent.com (uc3db145b7c034fad482258d867c.dl.dropboxusercontent.com)... 162.125

In [ ]:
url = 'https://raw.githubusercontent.com/saintsjd/sic4-list/master/sic-codes.csv'

# HTTP GET kérés küldése a fájl letöltéséhez
response = requests.get(url)

if response.status_code == 200:
    # A fájl tartalmának kiírása, ha a letöltés sikeres volt
    with open('sic-codes.csv', 'wb') as file:
        file.write(response.content)
    print('A fájl sikeresen letöltve.')
else:
    print('Hiba történt a letöltés során. Hibaüzenet:', response.status_code)

A fájl sikeresen letöltve.


In [ ]:
url = 'https://www2.census.gov/programs-surveys/cbp/technical-documentation/records-layouts/sic-code-descriptions/sic86_87.txt'

# HTTP GET kérés küldése a fájl letöltéséhez
response = requests.get(url)

if response.status_code == 200:
    # A fájl tartalmának kiírása, ha a letöltés sikeres volt
    with open('sic-codes2.csv', 'wb') as file:
        file.write(response.content)
    print('A fájl sikeresen letöltve.')
else:
    print('Hiba történt a letöltés során. Hibaüzenet:', response.status_code)



A fájl sikeresen letöltve.


In [ ]:
# Dataframe-ek betöltése
crsp = pd.read_csv("/content/CRSP_data_ready.csv", sep=",")
crsp = crsp[crsp.columns[1:]]
gc.collect() # memória takarítás
sp500 = pd.read_csv("/content/sp500_data_ready.csv", sep=",")
ff3 = pd.read_csv("/content/ff3_data_ready.csv", sep=",")
#sic = pd.read_csv("/content/sic-codes.csv", sep=',')

#sic = pd.read_csv("/content/sic-codes2.csv", sep='\t')
#sic["SIC"] = pd.to_numeric(sic['SIC'], errors='coerce')
#sic = sic.rename(columns={'NAME   *indicates codes used only in 1986': 'Description'})
#sic = sic[~sic['SIC'].isna()]
#sic['SIC'] = sic['SIC'].astype(int)
#sic.dtypes

###############################################################################
# Oké, én megint órákat szívtam érdemi eredmény nélkül, illet az érdemi eredmény az,
# hogy két forrást is találtam a SIC kódokra és persze egyik lista sem teljes, pedig
# mindkettő hivatalos forrásból van.
# Elegánsabb lenne mergelni és utána szűrni, de ezt elengedem most. Alább bemásol-
# tam azokat a SIC 4-digit kódokat, amikre futtatni szeretnénk, majd egyesével
# futtatunk rájuk. Bocs.

<ipython-input-5-3b72b31a479f>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp = pd.read_csv("/content/CRSP_data_ready.csv", sep=",")


In [ ]:
crsp.head()

,date,PERMNO,CUSIP,SIC,SH,EX,Prc,Tvol,Ret,Shr,AbsPrc,Size,lnRet
0,2000-01-03,10001,36720410,4920,11,3.0,8.5625,1721.0,0.007353,2450.0,8.5625,9.951236,0.007326
1,2000-01-04,10001,36720410,4920,11,3.0,8.4375,1080.0,-0.014599,2450.0,8.4375,9.936529,-0.014707
2,2000-01-05,10001,36720410,4920,11,3.0,8.5625,1711.0,0.014815,2450.0,8.5625,9.951236,0.014706
3,2000-01-06,10001,36720410,4920,11,3.0,8.5000,580.0,-0.007299,2450.0,8.5000,9.943909,-0.007326
4,2000-01-07,10001,36720410,4920,11,3.0,8.4375,1406.0,-0.007353,2450.0,8.4375,9.936529,-0.007380


In [ ]:
crsp_desc=crsp[["date", "PERMNO", "SIC", "Prc", "Shr", "Ret", "lnRet"]]
gc.collect()
crsp_desc.head()

,date,PERMNO,SIC,Prc,Shr,Ret,lnRet
0,2000-01-03,10001,4920,8.5625,2450.0,0.007353,0.007326
1,2000-01-04,10001,4920,8.4375,2450.0,-0.014599,-0.014707
2,2000-01-05,10001,4920,8.5625,2450.0,0.014815,0.014706
3,2000-01-06,10001,4920,8.5000,2450.0,-0.007299,-0.007326
4,2000-01-07,10001,4920,8.4375,2450.0,-0.007353,-0.007380


In [ ]:
crsp_desc_2002 = crsp_desc.loc[(crsp_desc["date"] >= "2002-01-01") & (crsp_desc["date"] <= "2002-12-31")].copy()
crsp_desc_2011 = crsp_desc.loc[(crsp_desc["date"] >= "2011-01-01") & (crsp_desc["date"] <= "2011-12-31")].copy()

In [ ]:
crsp_desc_2002 = crsp_desc_2002.loc[crsp["SIC"].isin([3482, 3483, 3484, 7381, 7382, 6311, 6321, 6324, 6331, 6351, 6361, 6371, 6399, 6531, 6798, 4512, 4513, 4522, 4581, 4724, 4725])].copy()
crsp_desc_2011 = crsp_desc_2011.loc[crsp["SIC"].isin([3482, 3483, 3484, 7381, 7382, 6311, 6321, 6324, 6331, 6351, 6361, 6371, 6399, 6531, 6798, 4512, 4513, 4522, 4581, 4724, 4725])].copy()

In [ ]:
#SP500 merge

crsp_desc_2002 = crsp_desc_2002.merge(
        sp500.rename(columns={"return_index": "sp500"})[["date", "sp500"]],
        how="inner",
        left_on="date",
        right_on="date"
    )

crsp_desc_2011 = crsp_desc_2011.merge(
        sp500.rename(columns={"return_index": "sp500"})[["date", "sp500"]],
        how="inner",
        left_on="date",
        right_on="date"
    )

# FF3 merge

crsp_desc_2002 = crsp_desc_2002.merge(
        ff3[["date", "Mkt-RF", "SMB", "HML", "RF"]],
        how="inner",
        left_on="date",
        right_on="date"
    )
crsp_desc_2002 = crsp_desc_2002.rename(columns={'Mkt-RF': 'MktRF'})

crsp_desc_2011 = crsp_desc_2011.merge(
        ff3[["date", "Mkt-RF", "SMB", "HML", "RF"]],
        how="inner",
        left_on="date",
        right_on="date"
    )
crsp_desc_2011 = crsp_desc_2011.rename(columns={'Mkt-RF': 'MktRF'})
  # Rendezés részvényenként, dátumra

crsp_desc_2002=crsp_desc_2002.sort_values(by=['PERMNO', 'date'])
crsp_desc_2011=crsp_desc_2011.sort_values(by=['PERMNO', 'date'])

In [ ]:
crsp_desc_2002.describe()

,PERMNO,SIC,Prc,Shr,Ret,lnRet,sp500,MktRF,SMB,HML,RF
count,35247.000000,35247.000000,35247.000000,3.524700e+04,35247.000000,35247.000000,35247.000000,35247.000000,35247.000000,35247.000000,35247.000000
mean,67489.941300,6076.633047,562.719735,1.006500e+05,-0.000401,-0.001269,-0.001064,-0.089072,0.016433,0.039462,0.006506
std,23218.438353,825.407865,6099.259675,2.466191e+05,0.042209,0.041792,0.016281,1.582385,0.667082,0.544891,0.000642
min,10085.000000,3482.000000,-185.490010,1.314000e+03,-0.679487,-1.137832,-0.042423,-3.950000,-2.470000,-2.110000,0.005000
25%,58077.000000,6311.000000,9.900000,1.237400e+04,-0.014101,-0.014201,-0.012094,-1.160000,-0.390000,-0.310000,0.006000
50%,76779.000000,6331.000000,21.950000,3.647300e+04,0.000000,0.000000,-0.001750,-0.120000,0.010000,0.070000,0.007000
75%,84406.000000,6331.000000,37.795000,9.794300e+04,0.012442,0.012365,0.007720,0.770000,0.490000,0.380000,0.007000
max,92655.000000,7382.000000,78300.000000,2.615492e+06,1.727273,1.003302,0.055744,5.430000,1.490000,1.900000,0.007000


In [ ]:
crsp_desc_2011.describe()

,PERMNO,SIC,Prc,Shr,Ret,lnRet,sp500,MktRF,SMB,HML,RF
count,34431.000000,34431.000000,34431.000000,3.443100e+04,34431.000000,34431.000000,34431.000000,34431.000000,34431.000000,34431.000000,34431.000000
mean,73519.504516,6126.819639,883.118887,1.467947e+05,-0.000195,-0.000788,0.000006,0.013905,-0.014276,-0.031174,0.000077
std,22243.720358,745.965097,9943.532559,2.484928e+05,0.034175,0.035046,0.014631,1.517605,0.655242,0.551090,0.000266
min,10822.000000,3483.000000,-28.125000,9.390000e+02,-0.839506,-1.829499,-0.068958,-6.970000,-1.910000,-1.630000,0.000000
25%,62885.000000,6311.000000,7.950000,2.004300e+04,-0.013553,-0.013646,-0.006879,-0.730000,-0.450000,-0.350000,0.000000
50%,81666.000000,6331.000000,20.830000,4.799800e+04,0.000000,0.000000,0.001001,0.120000,0.010000,-0.060000,0.000000
75%,90162.000000,6351.000000,40.970000,1.611680e+05,0.012232,0.012158,0.007357,0.750000,0.430000,0.310000,0.000000
max,93374.000000,7382.000000,131300.000000,1.899224e+06,0.730240,0.548260,0.046317,4.970000,3.590000,2.110000,0.001000


In [ ]:
crsp_desc_2002_stat=crsp_desc_2002.describe()
crsp_desc_2002_stat.to_excel('crsp_desc_2002_stat.xlsx', index=False)

crsp_desc_2011_stat=crsp_desc_2011.describe()
crsp_desc_2011_stat.to_excel('crsp_desc_2011_stat.xlsx', index=False)

In [ ]:
crsp=crsp[["date", "PERMNO", "SIC", "Size", "lnRet"]]
gc.collect()

0

3482	Small Arms Ammunition
3483	Ammunition, Except for Small Arms
3484	Small Arms

7381	Detective, Guard, and Armored Car Services
7382	Security Systems Services

6311	Life Insurance
6321	Accident and Health Insurance
6324	Hospital and Medical Service Plans
6331	Fire, Marine, and Casualty Insurance
6351	Surety Insurance
6361	Title Insurance
6371	Pension, Health, and Welfare Funds
6399	Insurance Carriers, Not Elsewhere Classified

6531	Real Estate Agents and Managers
6798	Real Estate Investment Trusts

4512 Air Transportation, Scheduled
4513 Air Courier Services
4522 Air Transportation, Nonscheduled
4581 Airports, Flying Fields, and Airport Terminal Services

4724 Travel Agencies
4725 Tour Operators

*dőlt szöveg*# innen a másik loopban van

In [ ]:
crsp_arms = crsp.loc[crsp["SIC"].isin([3482, 3483, 3484])].copy()
crsp_security = crsp.loc[crsp["SIC"].isin([7381, 7382])].copy()
crsp_insurance = crsp.loc[crsp["SIC"].isin([6311, 6321, 6324, 6331, 6351, 6361, 6371, 6399])].copy()
crsp_realest = crsp.loc[crsp["SIC"].isin([6531, 6798])].copy()
crsp_air = crsp.loc[crsp["SIC"].isin([4512, 4513, 4522, 4581])].copy()
crsp_travel = crsp.loc[crsp["SIC"].isin([4724, 4725])].copy()

crsp_arms_2011 = crsp_arms.loc[(crsp_arms["date"] >= "2011-01-01") & (crsp_arms["date"] <= "2011-12-31")].copy()
crsp_arms_2011["date"] = pd.to_datetime(crsp_arms_2011["date"]).dt.date
crsp_arms_2011

,date,PERMNO,SIC,Size,lnRet
9067519,2011-01-03,73219,3484,12.556814,-0.013166
9067520,2011-01-04,73219,3484,12.558138,0.001324
9067521,2011-01-05,73219,3484,12.562760,0.004622
9067522,2011-01-06,73219,3484,12.546154,-0.016606
9067523,2011-01-07,73219,3484,12.546824,0.000670
...,...,...,...,...,...
9943274,2011-12-23,76477,3483,14.450493,-0.002615
9943275,2011-12-27,76477,3483,14.445767,-0.004726
9943276,2011-12-28,76477,3483,14.428068,-0.017700
9943277,2011-12-29,76477,3483,14.440314,0.012246


In [ ]:
crsp_security_2011 = crsp_security.loc[(crsp_security["date"] >= "2011-01-01") & (crsp_security["date"] <= "2011-12-31")].copy()
crsp_security_2011["date"] = pd.to_datetime(crsp_security_2011["date"]).dt.date
crsp_security_2011


,date,PERMNO,SIC,Size,lnRet
4252005,2011-01-03,18649,7381,14.059540,0.023168
4252006,2011-01-04,18649,7381,14.052243,-0.007297
4252007,2011-01-05,18649,7381,14.069666,0.017423
4252008,2011-01-06,18649,7381,14.043786,-0.025881
4252009,2011-01-07,18649,7381,14.031898,-0.011887
...,...,...,...,...,...
23890489,2011-09-09,92328,7382,13.233792,0.009677
23890490,2011-09-12,92328,7382,13.232185,-0.001606
23890491,2011-09-13,92328,7382,13.225734,-0.006452
23890492,2011-09-14,92328,7382,13.260714,0.034980


In [ ]:
crsp_insurance_2011 = crsp_insurance.loc[(crsp_insurance["date"] >= "2011-01-01") & (crsp_insurance["date"] <= "2011-12-31")].copy()
crsp_insurance_2011["date"] = pd.to_datetime(crsp_insurance_2011["date"]).dt.date
crsp_insurance_2011

,date,PERMNO,SIC,Size,lnRet
484781,2011-01-03,10822,6351,10.641250,0.000000
484782,2011-01-04,10822,6351,10.669420,0.028170
484783,2011-01-05,10822,6351,10.669420,0.000000
484784,2011-01-06,10822,6351,10.645811,-0.023610
484785,2011-01-07,10822,6351,10.609316,-0.036494
...,...,...,...,...,...
24578100,2011-12-23,93374,6361,14.116951,-0.006995
24578101,2011-12-27,93374,6361,14.133966,0.017015
24578102,2011-12-28,93374,6361,14.101228,-0.027995
24578103,2011-12-29,93374,6361,14.107547,0.006319


In [ ]:
crsp_realest_2011 = crsp_realest.loc[(crsp_realest["date"] >= "2011-01-01") & (crsp_realest["date"] <= "2011-12-31")].copy()
crsp_realest_2011["date"] = pd.to_datetime(crsp_realest_2011["date"]).dt.date
crsp_realest_2011

,date,PERMNO,SIC,Size,lnRet
1434975,2011-01-03,12373,6798,14.537874,0.003303
1434976,2011-01-04,12373,6798,14.532364,-0.005509
1434977,2011-01-05,12373,6798,14.537507,0.005144
1434978,2011-01-06,12373,6798,14.542077,0.004570
1434979,2011-01-07,12373,6798,14.544445,0.002368
...,...,...,...,...,...
24017422,2011-12-23,92502,6531,13.220487,-0.003690
24017423,2011-12-27,92502,6531,13.213996,-0.006490
24017424,2011-12-28,92502,6531,13.184735,-0.025437
24017425,2011-12-29,92502,6531,13.203711,0.018976


In [ ]:
crsp_air_2011 = crsp_air.loc[(crsp_air["date"] >= "2011-01-01") & (crsp_air["date"] <= "2011-12-31")].copy()
crsp_air_2011["date"] = pd.to_datetime(crsp_air_2011["date"]).dt.date
crsp_air_2011

,date,PERMNO,SIC,Size,lnRet
4675609,2011-01-03,21020,4512,14.786147,0.016550
4675610,2011-01-04,21020,4512,14.808620,0.022473
4675611,2011-01-05,21020,4512,14.865024,0.056404
4675612,2011-01-06,21020,4512,14.850923,-0.014101
4675613,2011-01-07,21020,4512,14.897174,0.046251
...,...,...,...,...,...
23567632,2011-12-23,91926,4512,15.769489,-0.003594
23567633,2011-12-27,91926,4512,15.751318,-0.018171
23567634,2011-12-28,91926,4512,15.743956,-0.007362
23567635,2011-12-29,91926,4512,15.740255,-0.003702


In [ ]:
crsp_travel_2011 = crsp_travel.loc[(crsp_travel["date"] >= "2011-01-01") & (crsp_travel["date"] <= "2011-12-31")].copy()
crsp_travel_2011["date"] = pd.to_datetime(crsp_travel_2011["date"]).dt.date
crsp_travel_2011

,date,PERMNO,SIC,Size,lnRet
20713105,2011-01-03,89321,4725,12.263254,0.008658
20713106,2011-01-04,89321,4725,12.259800,-0.003454
20713107,2011-01-05,89321,4725,12.289627,0.029828
20713108,2011-01-06,89321,4725,12.251984,-0.037644
20713109,2011-01-07,89321,4725,12.234393,-0.017591
...,...,...,...,...,...
23751415,2011-12-23,92183,4724,12.882048,-0.018301
23751416,2011-12-27,92183,4724,12.882048,0.000000
23751417,2011-12-28,92183,4724,12.879406,-0.002642
23751418,2011-12-29,92183,4724,12.892546,0.013141


In [ ]:
event_date = pd.to_datetime("2011-09-11").date()
if event_date.isoweekday() in [6, 7]:
        event_date = (event_date + pd.offsets.BusinessDay()).date()
event_date
event_dates = [event_date + pd.offsets.BusinessDay(x) for x in range(-1, 6, 1)]
estimation_end = (event_date - pd.offsets.BusinessDay(22)).date()
estimation_start = (event_date - pd.offsets.BusinessDay(150)).date()
estimation_start

datetime.date(2011, 2, 14)

In [ ]:
print(crsp_air_2002["PERMNO"].unique())
print("részvények száma az elemzésben: "+ str(len(crsp_air_2002["PERMNO"].unique())))

[19596 21020 26112 28804 28847 34841 51721 58683 60628 64004 79641 80670
 80778 81666 82157 82235 83467 85732 87447 89353 89367]
részvények száma az elemzésben: 21


In [ ]:
j = 1

sic_values = [[3482, 3483, 3484],
              [7381, 7382],
              [6311, 6321, 6324, 6331, 6351, 6361, 6371, 6399],
              [6531, 6798],
              [4512, 4513, 4522, 4581],
              [4724, 4725]]
sic_labels = ["Arms, ammo", "Security, guard", "Insurance",
              "Real est. agent and op.", "Air transport", "Travel"]

yr = 2002

i = 0

for i, sic_list in enumerate(sic_values):

  #iparági szűrés
  ewp = crsp.loc[crsp["SIC"].isin(sic_list)].copy()

  #sp500 merge

  ewp = ewp.merge(
        sp500.rename(columns={"return_index": "market"})[["date", "market"]],
        how="inner",
        left_on="date",
        right_on="date"
    )

  # FF3 merge

  ewp = ewp.merge(
        ff3[["date", "Mkt-RF", "SMB", "HML", "RF"]],
        how="inner",
        left_on="date",
        right_on="date"
    )
  ewp = ewp.rename(columns={'Mkt-RF': 'MktRF'})

  # Rendezés részvényenként, dátumra

  ewp=ewp.sort_values(by=['PERMNO', 'date'])

  # évre szűrés

  #if j==0:
    #ewp_2002 = ewp.loc[(ewp["date"] >= "2002-01-01") & (ewp["date"] <= "2002-12-31")].copy()
    #ewp_2002["date"] = pd.to_datetime(ewp_2002["date"]).dt.date
    #ewportfolio=ewp_2002
  #elif j==1:
  ewp_2011 = ewp.loc[(ewp["date"] >= "2011-01-01") & (ewp["date"] <= "2011-12-31")].copy()
  ewp_2011["date"] = pd.to_datetime(ewp_2011["date"]).dt.date
  ewportfolio=ewp_2011
  # egyenlő súlyú portfolio hozam képzése

  ewportfolio['portflnret'] = ewportfolio.groupby('date')['lnRet'].transform('mean')
  ewportfolio.drop(columns=['lnRet'], inplace=True)
  ewportfolio.sort_values('date', ascending=True, inplace=True)
  ewportfolio = ewportfolio.drop_duplicates(subset='date', keep='first')
  ewportfolio['PERMNO'] = 'Portfolio'

  # OLS Market modellre és FF3 modellre, majd becslés, AR és t-stat számítás,
  # csv tábla generálás
  ewportfolio["estimation"] = np.where((ewportfolio["date"] >= estimation_start) & (ewportfolio["date"] <= estimation_end), 1, 0)
  ewportfolio["event"] = np.where(ewportfolio["date"].isin([x.date() for x in event_dates]), 1, 0)
  ewportfolio = ewportfolio[(ewportfolio["estimation"] == 1) | (ewportfolio["event"] == 1)]
  estimation = ewportfolio["estimation"] == 1
  model = sm.OLS(endog=ewportfolio[estimation]["portflnret"], exog=sm.tools.add_constant(ewportfolio[estimation]["market"]))
  results = model.fit()
  ewportfolio["estimated"] = ewportfolio.apply(lambda x: results.params.const + x["market"] * results.params.market, axis=1)
  ewportfolio["ar"] = ewportfolio["portflnret"] - ewportfolio["estimated"]
  #szigma
  sigma1nap = np.sqrt((1/ewportfolio[ewportfolio['estimation'] == 1].shape[0] * ((ewportfolio[ewportfolio['estimation'] == 1]['ar'] ** 2).sum())))
  #sigma1napff3 = np.sqrt((1/ewportfolio[ewportfolio['estimation'] == 1].shape[0] * ((ewportfolio[ewportfolio['estimation'] == 1]['arff3'] ** 2).sum())))
  # eseményablakra szűrés
  ewportfolio = ewportfolio.loc[ewportfolio["event"] == 1]
  ewportfolio["window"] = (pd.Series(range(ewportfolio.shape[0])) - 1).values
  sigmawin = np.sqrt((5+1)*sigma1nap**2)
  # kummulált hozam és tesztstatisztikák számítása (modellenként)
  ewportfolio["car"] = np.where(ewportfolio['window'] == 5, (ewportfolio[ewportfolio['window'] > 0]['ar']).sum(), np.nan)
  ewportfolio['TS'] = np.where(ewportfolio['window'] < 1, ewportfolio['ar'] / sigma1nap, np.nan)
  ewportfolio['TS'] = np.where(ewportfolio['window'] == 5, (ewportfolio[ewportfolio['window'] > 0]['ar']).sum() / sigmawin, ewportfolio['TS'])
  #ewportfolio["carff3"] = np.where(ewportfolio['window'] == 5, (ewportfolio[ewportfolio['window'] > 0]['arff3']).sum(), np.nan)
  #ewportfolio['TSff3'] = np.where(ewportfolio['window'] < 1, ewportfolio['arff3'] / sigma1napff3, np.nan)
  #ewportfolio['TSff3'] = np.where(ewportfolio['window'] == 5, (ewportfolio[ewportfolio['window'] > 0]['arff3']).sum() / sigmawinff3, ewportfolio['TSff3'])
  ewportfolio.to_csv('0' + str(i) + '_' + str(2011) + '_' + sic_labels[i] + '.csv', index=False, sep=";")
  i = i + 1


<ipython-input-18-b62438726845>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ewportfolio['PERMNO'] = 'Portfolio'
<ipython-input-18-b62438726845>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ewportfolio["estimation"] = np.where((ewportfolio["date"] >= estimation_start) & (ewportfolio["date"] <= estimation_end), 1, 0)
<ipython-input-18-b62438726845>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [ ]:
sic_values = [[3482, 3483, 3484],
              [7381, 7382],
              [6311, 6321, 6324, 6331, 6351, 6361, 6371, 6399],
              [6531, 6798],
              [4512, 4513, 4522, 4581],
              [4724, 4725]]
sic_labels = ["Arms, ammo", "Security, guard", "Insurance",
              "Real est. agent and op.", "Air transport", "Travel"]

yr = 2002

i = 0

for i, sic_list in enumerate(sic_values):

  # Iparági szűrés
  vwp = crsp.loc[crsp["SIC"].isin(sic_list)].copy()

  # S&P500 merge-ölés
  vwp = vwp.merge(
          sp500.rename(columns={"return_index": "market"})[["date", "market"]],
          how="inner",
          left_on="date",
          right_on="date"
      )

  # FF3 faktorok merge-ölés
  vwp = vwp.merge(
          ff3[["date", "Mkt-RF", "SMB", "HML", "RF"]],
          how="inner",
          left_on="date",
          right_on="date"
      )

  # Rendezés részvényenként, dátumra
  vwp=vwp.sort_values(by=['PERMNO', 'date'])

  # évre szűrés
  yrstart = str(yr) + "-01-01"
  yrend = str(yr) + "-12-31"
  vwp = vwp.loc[(vwp["date"] >= yrstart) & (vwp["date"] <= yrend)].copy()
  vwp["date"] = pd.to_datetime(vwp["date"]).dt.date

  # Value weighted portfolio képzése
  vwp['weight'] = vwp['Size']/vwp.groupby('date')['Size'].transform('sum')
  vwp['portflnret'] = vwp['lnRet']*vwp['weight']

  vwp['portflnret'] = vwp.groupby('date')['portflnret'].transform('sum')
  vwp.drop(columns=['weight', 'lnRet'], inplace=True)
  vwp.sort_values('date', ascending=True, inplace=True)
  vwp = vwp.drop_duplicates(subset='date', keep='first')
  vwp['PERMNO'] = 'Portfolio'

  # becslési és eseményablak számítása,
  # most itt megelőző naptól t+5-ig
  # becslési pedig 150 munkanappal korábbtól 22 munkanappal korábbig
  event_date = pd.to_datetime(str(yr) + "-09-11").date()
  if event_date.isoweekday() in [6, 7]:
          event_date = (event_date + pd.offsets.BusinessDay()).date()
  event_date
  event_dates = [event_date + pd.offsets.BusinessDay(x) for x in range(-1, 6, 1)]
  estimation_end = (event_date - pd.offsets.BusinessDay(22)).date()
  estimation_start = (event_date - pd.offsets.BusinessDay(150)).date()

  # OLS Market modellre és FF3 modellre, majd becslés, AR és t-stat számítás,
  # és mindezt beírva táblába
  vwp["estimation"] = np.where((vwp["date"] >= estimation_start) & (vwp["date"] <= estimation_end), 1, 0)
  vwp["event"] = np.where(vwp["date"].isin([x.date() for x in event_dates]), 1, 0)
  vwp = vwp[(vwp["estimation"] == 1) | (vwp["event"] == 1)]
  estimation = vwp["estimation"] == 1
  # Market modell
  model = sm.OLS(endog=vwp[estimation]["portflnret"], exog=sm.tools.add_constant(vwp[estimation]["market"]))
  results = model.fit()
  vwp["estimated"] = vwp.apply(lambda x: results.params.const + x["market"] * results.params.market, axis=1)
  vwp["ar"] = vwp["portflnret"] - vwp["estimated"]
  vwp["portflnret"] - vwp["estimated"]
  # FF3 modell
  vwp = vwp.rename(columns={'Mkt-RF': 'MktRF'})
  modelff3 = sm.OLS(endog=vwp[estimation]["portflnret"], exog=sm.tools.add_constant(vwp[estimation][["MktRF", "SMB", "HML"]]))
  resultsff3 = modelff3.fit()
  vwp["estimatedff3"] = vwp.apply(lambda x: results.params.const + x["MktRF"] * resultsff3.params.MktRF +
                                  x['SMB'] * resultsff3.params.SMB + x['HML'] * resultsff3.params.HML, axis=1)
  vwp["arff3"] = vwp["portflnret"] - vwp["estimatedff3"]
  # szigma
  sigma1nap = np.sqrt((1/vwp[vwp['estimation'] == 1].shape[0] * ((vwp[vwp['estimation'] == 1]['ar'] ** 2).sum())))
  sigma1napff3 = np.sqrt((1/vwp[vwp['estimation'] == 1].shape[0] * ((vwp[vwp['estimation'] == 1]['arff3'] ** 2).sum())))
  # itt leszűrűnk az eseményablakra
  vwp = vwp.loc[vwp["event"] == 1]
  # majd T1..T5 kummulált AR-t számolunk, Tetlock eljárásához hasonlóan
  # (vagyis ha lenne T-1 vagy T0 AR, akkor T+5-re korrigálna)
  vwp["window"] = (pd.Series(range(vwp.shape[0])) - 1).values
  sigmawin = np.sqrt((5+1)*sigma1nap**2)
  sigmawinff3 = np.sqrt((5+1)*sigma1napff3**2)
  # kummulált hozam és tesztstatisztikák számítása (modellenként)
  vwp["car"] = np.where(vwp['window'] == 5, (vwp[vwp['window'] > 0]['ar']).sum(), np.nan)
  vwp['TS'] = np.where(vwp['window'] < 1, vwp['ar'] / sigma1nap, np.nan)
  vwp['TS'] = np.where(vwp['window'] == 5, (vwp[vwp['window'] > 0]['ar']).sum() / sigmawin, vwp['TS'])
  vwp["carff3"] = np.where(vwp['window'] == 5, (vwp[vwp['window'] > 0]['arff3']).sum(), np.nan)
  vwp['TSff3'] = np.where(vwp['window'] < 1, vwp['arff3'] / sigma1napff3, np.nan)
  vwp['TSff3'] = np.where(vwp['window'] == 5, (vwp[vwp['window'] > 0]['arff3']).sum() / sigmawinff3, vwp['TSff3'])
  vwp.to_csv('0' + str(i) + '_' + str(yr) + '_' + sic_labels[i] + '.csv', index=False, sep=";")
  i = i + 1